In [1]:
## Use Trained model to generate text

## Anaconda environment setup as follows:
#	*** create new environment
#conda create -n --newENV python=3.7
#
#	*** general stuff
#conda install jupyter
#
#	*** Story Generation
#conda install numpy
#conda install requests
#conda install regex
#pip install tensorflow==1.15.2
#** not done this so far pip install toposort
#pip install gpt-2-simple			## Successfully installed gpt-2-simple-0.7.1 regex-2020.10.23 toposort-1.5

In [1]:
import os
import json

import gpt_2_simple as gpt2
import tensorflow as tf

from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
from gpt_2_simple.src.load_dataset import load_dataset, Sampler
from gpt_2_simple.src.accumulate import AccumulatingOptimizer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
assert tf.__version__ < '2.0.0', "gpt-2-simple currently does not support " \
    "TensorFlow 2.0. You'll need to use a virtualenv/cloud computer which " \
    "has Tensorflow 1.X on it."

In [4]:
###############################################################################################################
## -------------------     STORY GENERATOR STARTS                  STORY GENERATOR STARTS   -------------------
###############################################################################################################
def my_load_gpt2(_sess,
              _checkpoint='latest',
              _run_name="run1",
              _checkpoint_dir="checkpoint",
              _model_name=None,
              _model_dir='models',
              _multi_gpu=False):
    """Loads the model checkpoint or existing model into a TensorFlow session for repeated predictions.
    """

    if _model_name:
        checkpoint_path = os.path.join(_model_dir, _model_name)
    else:
        checkpoint_path = os.path.join(_checkpoint_dir, _run_name)
    print(f"\ncheckpoint_path = {checkpoint_path}\n")

    hparams = model.default_hparams()
    with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    context = tf.compat.v1.placeholder(tf.int32, [1, None])

    gpus = []
    if _multi_gpu:
        gpus = get_available_gpus()

    output = model.model(hparams=hparams, X=context, gpus=gpus)

    if _checkpoint=='latest':
        ckpt = tf.train.latest_checkpoint(checkpoint_path)
    else:
        ckpt = os.path.join(checkpoint_path,_checkpoint)

    saver = tf.compat.v1.train.Saver(allow_empty=True)
    _sess.run(tf.compat.v1.global_variables_initializer())

    if _model_name:
        print(f"\nLoading pretrained model :: {ckpt}\n")
    else:
        print(f"\nLoading checkpoint :: {ckpt}\n")
    saver.restore(_sess, ckpt)

def reset_session(_sess, threads=-1, server=None):
    """Resets the current TensorFlow session, to clear memory
    or load another model.
    """

    tf.compat.v1.reset_default_graph()
    _sess.close()

def gen_inference_functionality(_gpt_params):
    _seed, _LENGTH, _TEMPERATURE, _NSAMPLES, _BATCH_SIZE, _RUN_NAME = _gpt_params
    
    ## set up the pre-trained GPT-2 model from checkpoint directory
    
    ## copy checkpoint directory from gdrive - not required as saved it locally
    ##gpt2.copy_checkpoint_from_gdrive(run_name='run1')
    GPT2_CHECKPOINT_DIR = r'/home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/'
    #RUN_NAME = r"run1"

    sess = gpt2.start_tf_sess()
    print(f"\nTF session started\n")
    #gpt2.load_gpt2(sess, run_name='run1')
    my_load_gpt2(
        _sess = sess,
        _checkpoint='latest',
        _run_name=_RUN_NAME,
        _checkpoint_dir=GPT2_CHECKPOINT_DIR,
        _model_name=None,
        _model_dir='models',
        _multi_gpu=False)
    
    print(f"\n\nGenerating using 'prefix' = {_seed}\n")
    gpt2.generate(sess,
                  length=_LENGTH,
                  temperature=_TEMPERATURE,
                  prefix=_seed,
                  nsamples=_NSAMPLES,
                  batch_size=_BATCH_SIZE,
                  checkpoint_dir=GPT2_CHECKPOINT_DIR
                 )
    
    reset_session(sess)
    print(f"\n\nFINISHED\n")

    return

###############################################################################################################
## ---------------------     STORY GENERATOR ENDS                  STORY GENERATOR ENDS   ---------------------
###############################################################################################################

In [5]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
LENGTH = 300
TEMPERATURE = 0.95
NSAMPLES = 1
BATCH_SIZE = 1
RUN_NAME = r"Run2_File11_2_checkpoint_run2"

## seed value
SEED_STRING = r'they had a couple of beers and '
seed_string = SEED_STRING.capitalize()

## check samples and batch size values are compatible
assert (NSAMPLES % BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={NSAMPLES} % BATCH_SIZE={BATCH_SIZE} != 0"

GPT_PARAMS = (seed_string, LENGTH, TEMPERATURE, NSAMPLES, BATCH_SIZE, RUN_NAME)

gen_inference_functionality(GPT_PARAMS)


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating using 'prefix' = They had a couple of beers and 

They had a couple of beers and iced tea.
They were all, except Bailey, and can hardly even get the paper round.
Poor Bailey!
Poor Bailey!
It was a wretched pitiful thing to be watching for him -- but he did not care.
He was all in.
He was a little stiff, and he could not move, only you see he could not dig.
He did not dare to ask.
He had been out all day.
This Sandy George was hard to find, I suppose, when he was so drunk.
He could be seen waving his hands and waving his arm.
They were all laughing, and it was all the show, save the topic of Tommy Robber 's dying in